**Curso de Inteligencia Artificial y Aprendizaje Profundo**


# Introducción al aprendizaje reforzado

## El problema del bandido multibrazo

##  Autores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Oleg Jarma, ojarmam@unal.edu.co
4. Maria del Pilar Montenegro, pmontenegro88@gmail.com

## Contenido

* [Introducción](#Introducción)
* [El problema del bandido multibrazo](#El-problema-del-bandido-multibrazo)
* [Política ε-voraz](#Política-ε-voraz)
* [Implementación](#Implementación)


## Referencias

1. Adaptado de Markel Sanz, [Introducción al aprendizaje por refuerzo](https://medium.com/@markelsanz14/introducci%C3%B3n-al-aprendizaje-por-refuerzo-parte-1-el-problema-del-bandido-multibrazo-afe05c0c372e)
2. Sutton, R. S., & Barto, A. G. (2018). [Reinforcement learning: An introduction. MIT press](https://web.stanford.edu/class/psych209/Readings/SuttonBartoIPRLBook2ndEd.pdf).

## Introducción

Aprendizaje reforzado (Reinforcement Learning).

Ésta es una técnica de inteligencia artificial (IA) en la que un agente inteligente (**agent**) tiene que interactuar con un entorno (**environment**), escogiendo una de las acciones (**action**) que el entorno ofrece en cada uno de los posibles estados (**state**), e intentar conseguir la mayor recompensa (**reward**) posible a través de esas acciones.


Al principio, el agente no conoce nada sobre el entorno, por lo que tomará acciones de forma aleatoria. 

Si una acción trae una recompensa positiva, el agente deberá aprender a escoger esa acción más frecuentemente, mientras que si una acción trae una recompensa negativa, el agente deberá aprender a escoger esa acción menos frecuentemente. 

Así, el *agente aprenderá a escoger las acciones que maximicen la suma de recompensas recibidas*, también conocida como el retorno (**return**).

## El problema del bandido multibrazo 

Multi-armed bandit problem.

El problema del bandido multibrazo se puede ver como un problema de máquinas tragamonedas en un casino. 

Si tenemos un número $N$  de tragamonedas una nos da una recompensa positiva con probabilidad $p$, y ninguna recompensa con probabilidad $(1-p)$, ¿podemos crear un agente que maximice las recompensas escogiendo jugar siempre en la tragamonedas que más beneficio nos vaya a proporcionar? 


Pues la idea es la misma; tenemos un bandido con $N$ brazos, y cada brazo tiene una probabilidad distinta de darnos una recompensa positiva. El objetivo es crear un agente que maximice esas recompensas.


Para esta lección , usaremos un bandido de $N=5$ brazos (5-armed bandit). 

Éstas serán las probabilidades de cada brazo de dar una recompensa positiva: $[0.1, 0.3, 0.05, 0.55, 0.4]$. 


Como podemos ver, la mejor acción entre estas cinco es tirar del cuarto brazo. Sin embargo, el agente no dispone de esta información. 

Por lo tanto, deberá probar a tirar de todos los brazos varias veces, e ir aprendiendo cuál de todos es el mejor. Cuando vaya acumulando más información, empezará a tomar mejores decisiones, y a recibir mejores recompensas más frecuentemente.



<figure>
<center>
<img src="../Imagenes/Tragamonedas.jpeg" width="200" height="200" align="center"/>
</figure>

Fuente: [Introducción al aprendizaje por refuerzo](https://medium.com/@markelsanz14/introducci%C3%B3n-al-aprendizaje-por-refuerzo-parte-1-el-problema-del-bandido-multibrazo-afe05c0c372e)

## Política ε-voraz 

**ε-greedy policy**

Ésta será la política (*policy*) que decidirá qué acciones toma nuestro agente. La política es una función de probabilidad asociada a la siguiente acción que tomará el agente dado que se encuentra en el estado actual $s$.

La **política ε-voraz** consiste en que el agente casi siempre tomará la mejor acción posible dada la información que posee. 

Sin embargo, de vez en cuando, con una **probabilidad de ε, el agente tomará una acción completamente al azar**. 

De esta forma, si tras la primera acción el agente ha obtenido una recompensa positiva, no se quedará atascado escogiendo esa misma acción todo el rato.

Con probabilidad ε el agente explorará otras opciones.

Este valor ε lo decidiremos nosotros, y será la forma que tengamos de equilibrar el problema de exploración y explotación (exploration vs. exploitation). 

La **exploración** consiste en explorar todas las acciones posibles varias veces para ver cuál es la mejor, a pesar de que durante esa exploración no obtengamos recompensas muy buenas. 

La **explotación** consiste en maximizar las recompensas, por lo que el agente escogerá la mejor de las acciones cada vez. 

Por ello, es importante **equilibrar la exploración y la explotación**: si solo exploramos dos de las cinco acciones posibles, no sabremos si las acciones que nunca hemos probado nos traerán mayores recompensas, por lo que la exploración es necesaria; y sin embargo, si nos pasamos todo el rato explorando todas las opciones una y otra vez, nunca utilizaremos ese conocimiento para poder escoger la mejor acción y conseguir la mayor recompensa posible.

## Implementación

Crearemos un agente que resuelva el problema del bandido multibrazo. 

Empecemos con el código. Vamos a escribir una función que nos permita escoger uno de los cinco brazos, y nos devuelva una recompensa, dependiendo de la probabilidad de dicho brazo.

In [4]:
import numpy as np

def pull_bandit_arm(bandits, bandit_number):
    """Pull arm in position bandit_number and return the obtained reward."""
    result = np.random.uniform()
    # return a binary signal: (1 = reward; 0 = no reward)
    # with probability bandits[bandit_number] obtain a reward.
    return int(result <= bandits[bandit_number])

Ahora, crearemos la función que decide qué acción debe tomar el agente. Con probabilidad *epsilon* tomará una acción aleatoria; y si no, tomará la acción con mejor media de recompensas.


In [5]:
def take_epsilon_greedy_action(epsilon, average_rewards):
    """Take random action with probability epsilon, else take best action."""
    result = np.random.uniform()
    # exploration
    if result < epsilon:
        return np.random.randint(0, len(average_rewards)) # Random action
    # explotation
    else:
        return np.argmax(average_rewards) # Greedy action

Por último, definamos las probabilidades de los brazos como hemos mencionado anteriormente, y definamos los parámetros epsilon y la cantidad de iteraciones o acciones que vamos a tomar. 

Definamos también tres listas donde guardaremos información sobre las acciones ejecutadas hasta este momento y las recompensas conseguidas para cada brazo.


Ejecutamos el algoritmo llamando a las funciones, y guardamos las recompensas.

In [13]:
# Probability of success of each bandit
bandits = [0.1, 0.3, 0.05, 0.55, 0.4]
num_iterations = 1000 # episodes
epsilon = 0.1
# ecch episode finish when the agent pull the aro
# end receive a reward(which could be 

# seed for reproductibility
np.random.seed(600)

# Store info to know which one is the best action in each moment
# four lists of size len(bandits)
total_rewards = [0 for _ in range(len(bandits))]
total_attempts = [0 for _ in range(len(bandits))]
average_rewards = [0.0 for _ in range(len(bandits))]

for iteration in range(num_iterations+1):
    # select an action: select an arm
    action = take_epsilon_greedy_action(epsilon, average_rewards)
    # inform the action to the environment and receive a reward
    reward = pull_bandit_arm(bandits, action)
  
    # Store result
    # acumulate the reward for this arm
    total_rewards[action] += reward 
    # acumulate the attemps of this arm
    total_attempts[action] += 1  # for this arm
    # compute the current average reward 
    average_rewards[action] = total_rewards[action] / float(total_attempts[action])
  
    if iteration % 100 == 0:
        print('Average reward for bandits in iteration {} is {}'.format(iteration,
                                  ['{:.2f}'.format(elem) for elem in average_rewards]))

# Print results
best_bandit = np.argmax(average_rewards)
print('\nBest bandit is {} with an average observed reward of {:.4f}'
      .format(best_bandit, average_rewards[best_bandit]))
print('Total observed reward in the {} episodes has been {}'
      .format(num_iterations, sum(total_rewards)))


Average reward for bandits in iteration 0 is ['0.00', '0.00', '0.00', '0.00', '0.00']
Average reward for bandits in iteration 100 is ['0.14', '0.23', '0.00', '0.00', '0.00']
Average reward for bandits in iteration 200 is ['0.15', '0.27', '0.00', '0.00', '0.00']
Average reward for bandits in iteration 300 is ['0.18', '0.29', '0.00', '0.25', '0.27']
Average reward for bandits in iteration 400 is ['0.17', '0.29', '0.12', '0.53', '0.29']
Average reward for bandits in iteration 500 is ['0.16', '0.29', '0.10', '0.58', '0.32']
Average reward for bandits in iteration 600 is ['0.15', '0.29', '0.07', '0.55', '0.30']
Average reward for bandits in iteration 700 is ['0.14', '0.29', '0.07', '0.55', '0.26']
Average reward for bandits in iteration 800 is ['0.14', '0.29', '0.06', '0.57', '0.28']
Average reward for bandits in iteration 900 is ['0.14', '0.29', '0.05', '0.57', '0.27']
Average reward for bandits in iteration 1000 is ['0.13', '0.29', '0.05', '0.56', '0.26']

Best bandit is 3 with an average

Al final, vemos que el algoritmo escribe esto:

Tenemos la lista con la media de recompensas producidas por cada brazo, la cual se parece bastante a las probabilidades que hemos definido para cada uno de ellos. 

Es decir, con 1000 iteraciones hemos encontrado la probabilidad de éxito de cada brazo, y ahora sabemos que el cuarto brazo es el mejor (índice 3 en la lista). 

En este proceso, hemos obtenido 436 recompensas positivas. Es un número muy alto, teniendo en cuenta que escogiendo el mejor brazo desde el principio hubiéramos obtenido 550 recompensas positivas (porque tiene una probabilidad de 0.55). Podemos cambiar el valor de epsilon y la cantidad de iteraciones, para ver cómo estos valores afectan a la recompensa total recibida.
